In [1]:
from optimum.onnxruntime import ORTModelForQuestionAnswering, ORTModelForAudioClassification, ORTModelForSequenceClassification, ORTModelForSpeechSeq2Seq
from transformers import AutoConfig, Wav2Vec2Processor, AutoTokenizer, PretrainedConfig, WhisperProcessor
import time
import librosa
import numpy as np
import pandas as pd
import os

import audio_base 
import metric
import config

# Речь в текст

In [2]:

sampling_rate =  16000
path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_text = audio_base.SpeechtoText('whisper-tiny_onnx', "openai/whisper-base", "openai/whisper-tiny")
model_text.run(speech)

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


[' Слушай, я потратил обретьом кучу денег для того, чтобы притоситься в эту дару. Это что вообще такое? Посмотри на официантов, они все в черных каких-то рубашках с кислыми минами. Даже никто из них до сих пор не подошел к нам.']

In [3]:
path_a ='01_happiness_anger a_020.wav'
speech, sr = librosa.load(path_a, sr=sampling_rate)
model_em = audio_base.SpeechtoEmotion("wav2vec2-xls-r-300m-emotion-ru_onnx", "KELONMYOSA/wav2vec2-xls-r-300m-emotion-ru")
model_em.run(speech)

Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


'angry'

# Текст в эмоции

In [4]:
model_textem = audio_base.TexttoEmotion("rubert-tiny2-russian-emotion-detection_onnx", "Djacon/rubert-tiny2-russian-emotion-detection")
model_textem.run("ой беда, какая беда")

'sadness'

## Общий поток распознавания

In [5]:
path_a ='01_happiness_anger a_020.wav'
# path_a = '1725.wav'


# создаем модели

model_text = audio_base.SpeechtoText(config.model_name_speech_to_text[0], config.model_name_speech_to_text[1], config.model_name_speech_to_text[2])
model_textem = audio_base.TexttoEmotion(config.model_name_text_to_emo[0], config.model_name_text_to_emo[1])
model_em = audio_base.SpeechtoEmotion(config.model_name_speech_to_emo[0],config.model_name_speech_to_emo[1],)

# длина эпизода записи в с. 
dt = 1
d = 16000 * dt
n_d = 4


# rez_text_all = model_text.run(speech)
# rez_text_all_buf = rez_text_all[0] 
# rez_text_all_list = rez_text_all[0].split(' ')

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.
You are using a model of type whisper to instantiate a model of type . This is not supported for all configurations of models and can yield errors.


Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize
Ignored unknown kwarg option normalize


In [6]:
s, df =  audio_base.check_model(path_a, dt=1, d=d, n_d=4,  model_text=model_text, model_textem=model_textem, model_em=model_em)
s, df.head()

(' Слушай, потратил об этом  кучу денег для  того, что  притащится в эту дару. Это что   вообще такое? Посмотри на офицантов. Они все в    черных каких-то рубашках с   кислыми минами. Даже   никто из них до сих пор не  подошел к нам.',
      t                                         sub_string        word  \
 0  0.0   Слушай, потратил об этом кучу денег. Для того...     Слушай,   
 1  0.5   кучу денег для того, чтобы притащиться в эту ...        кучу   
 2  1.0   Для того, что притащится в эту дару, это что ...       того,   
 3  1.5   что притащится в эту дару. Это что вообще так...  притащится   
 4  2.0   В эту дару это что вообще такое посовинайся ц...      вообще   
 
           em1    em2  emotion  k2  k1  
 0     neutral  angry     0.50  26   0  
 1     neutral  angry     0.50  16   0  
 2     neutral  angry     0.50  15   4  
 3  enthusiasm  angry     0.55  36   4  
 4  enthusiasm  angry     0.55  20  19  )

In [8]:
s

' Слушай, потратил об этом  кучу денег для  того, что  притащится в эту дару. Это что   вообще такое? Посмотри на офицантов. Они все в    черных каких-то рубашках с   кислыми минами. Даже   никто из них до сих пор не  подошел к нам.'

In [9]:
dir_rez = 'save_rez/'

df.to_csv(dir_rez + '.'.join(path_a.split('/')[-1].split('.')[:-1]) + '.csv')

In [8]:
dir_resd = '../dataset/RESD_csv/'

dir_rez = 'save_rez/'

df_test = pd.read_csv(dir_resd + 'test.csv')
df_test.shape

(280, 4)

In [9]:
set(df_test.emotion.values)

{'anger', 'disgust', 'enthusiasm', 'fear', 'happiness', 'neutral', 'sadness'}

In [10]:
result =  []
for i in range(df_test.shape[0]):
        
    try:
        path_a = dir_resd + 'test/' +df_test.iloc[i,1]
        text = df_test.iloc[i,3]
        emotion = df_test.iloc[i,2]
        s, df =  audio_base.check_model(path_a, dt=1, d=d, n_d=4,  model_text=model_text, model_textem=model_textem, model_em=model_em)
        met = metric.calculate_wer(text, s)
        rez_emo = df.iloc[:,5].mean()
        
        df.to_csv(dir_rez + '.'.join(path_a.split('/')[-1].split('.')[:-1]) + '.csv')
        result.append([ i, path_a, text, s, emotion, rez_emo, met])
        print(end='.')    
    except: 
        print('error')

......

2024-04-26 16:55:47.458179040 [E:onnxruntime:, sequential_executor.cc:514 ExecuteKernel] Non-zero status code returned while running Conv node. Name:'/wav2vec2/feature_extractor/conv_layers.0/conv/Conv' Status Message: Invalid input shape: {7}


no audio or no convert
error
.no audio or no convert
error
no audio or no convert
error
..

2024-04-26 16:56:20.654958616 [E:onnxruntime:, sequential_executor.cc:514 ExecuteKernel] Non-zero status code returned while running Conv node. Name:'/wav2vec2/feature_extractor/conv_layers.6/conv/Conv' Status Message: Invalid input shape: {1}


no audio or no convert
error


...

no audio or no convert
error
...no audio or no convert
error
....no audio or no convert
error
.no audio or no convert
error
.....................no audio or no convert
error
.........no audio or no convert
error
no audio or no convert
error
.no audio or no convert
error
..no audio or no convert
error
....no audio or no convert
error
......no audio or no convert
error
no audio or no convert
error
........no audio or no convert
error
....no audio or no convert
error
..no audio or no convert
error
no audio or no convert
error
........................no audio or no convert
error
....no audio or no convert
error
.no audio or no convert
error
......no audio or no convert
error
no audio or no convert
error
no audio or no convert
error
no audio or no convert
error
..no audio or no convert
error
...

2024-04-26 17:14:59.175003497 [E:onnxruntime:, sequential_executor.cc:514 ExecuteKernel] Non-zero status code returned while running Conv node. Name:'/wav2vec2/feature_extractor/conv_layers.6/conv/Conv' Status Message: Invalid input shape: {1}


no audio or no convert
error
.no audio or no convert
error
..no audio or no convert
error
.no audio or no convert
error
no audio or no convert
error
no audio or no convert
error
.......

2024-04-26 17:16:09.407100684 [E:onnxruntime:, sequential_executor.cc:514 ExecuteKernel] Non-zero status code returned while running Conv node. Name:'/wav2vec2/feature_extractor/conv_layers.3/conv/Conv' Status Message: Invalid input shape: {2}


no audio or no convert
error
...no audio or no convert
error
no audio or no convert
error
...

In [11]:
pd.DataFrame(result, columns= ['No', 'name','text_true', 'text_recognize', 'emo_true', 'met' ]).to_csv('wer_analis_resd.csv')

ValueError: 6 columns passed, passed data had 7 columns